In [2]:
import cv2
import numpy as np
import joblib

# Load the trained model
model = joblib.load("PlasticDetection.pkl")

# Define the function to process each frame from the camera
def process_frame(frame):
    # Preprocess the frame
    frame = cv2.resize(frame, (100, 100))
    frame = frame.reshape(1, -1)
    frame = frame.astype('float32') / 255

    # Predict the label of the frame
    label = model.predict(frame)

    # Draw a rectangle and label on the frame indicating the prediction
    if label[0] == "plastic":
        cv2.rectangle(frame, (0, 0), (frame.shape[1], frame.shape[0]), (0, 255, 0), 2)
        cv2.putText(frame, "Plastic", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
    else:
        cv2.rectangle(frame, (0, 0), (frame.shape[1], frame.shape[0]), (0, 0, 255), 2)
        cv2.putText(frame, "Not Plastic", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

    return frame

# Set the IP address and port number of the camera stream on your mobile device
ip_address = '192.168.0.103'
port_number = '8080'

# Set the URL for the camera stream using the IP address and port number
url = f'http://{ip_address}:{port_number}/video'

# Open the mobile camera
cap = cv2.VideoCapture(0)
cap.open(url)

# Loop through the frames from the camera and process each frame
while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Process the frame
    processed_frame = process_frame(frame)

    # Display the processed frame
    cv2.imshow('frame', processed_frame)

    # Check for user input to exit the loop
    if cv2.waitKey(1) == ord('q'):
        break

# Release the camera and close all windows
cap.release()
cv2.destroyAllWindows()


In [3]:
import urllib.request
import cv2
import numpy as np
import joblib

# Load the trained model
model = joblib.load('PlasticDetection.pkl')

# Set the URL of the IP Webcam app on your mobile phone
url = 'http://192.168.0.103:8080/video'

# Open the camera stream
stream = urllib.request.urlopen(url)

# Loop through the frames in the camera stream
while True:
    # Read a frame from the camera stream
    img_bytes = stream.read()
    img_array = np.array(bytearray(img_bytes), dtype=np.uint8)
    img = cv2.imdecode(img_array, -1)
    
    # Preprocess the image
    img = cv2.resize(img, (100, 100))
    img = img.astype('float32') / 255
    img = img.reshape(1, -1)
    
    # Make a prediction with the model
    prediction = model.predict(img)
    
    # Display the result
    if prediction == 'plastic':
        print('Plastic detected!')
    else:
        print('Not plastic.')
        
    # Wait for a key press and exit if 'q' is pressed
    if cv2.waitKey(1) == ord('q'):
        break

# Release the camera stream
stream.release()
cv2.destroyAllWindows()


KeyboardInterrupt: 

In [13]:
import cv2
import winsound
cam = cv2.VideoCapture(1)
while cam.isOpened():
    ret, frame1 = cam.read()
    ret, frame2 = cam.read()
    diff = cv2.absdiff(frame1, frame2)
    gray = cv2.cvtColor(diff, cv2.COLOR_RGB2GRAY)
    blur = cv2.GaussianBlur(gray, (5, 5), 0)
    _, thresh = cv2.threshold(blur, 20, 255, cv2.THRESH_BINARY)
    dilated = cv2.dilate(thresh, None, iterations=3)
    contours, _ = cv2.findContours(dilated, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    # cv2.drawContours(frame1, contours, -1, (0, 255, 0), 2)
    for c in contours:
        if cv2.contourArea(c) < 5000:
            continue
        x, y, w, h = cv2.boundingRect(c)
        cv2.rectangle(frame1, (x, y), (x+w, y+h), (0, 255, 0), 2)
        winsound.PlaySound('alert.wav', winsound.SND_ASYNC)
    if cv2.waitKey(10) == ord('q'):
        break
    cv2.imshow('Granny Cam',frame1)

: 

Rectangle frames

In [1]:
import cv2
import numpy as np
from joblib import load

# Load the SVM model from the pickle file
model = load('PlasticDetection.pkl')

# Define the boundaries of the plastic bottle (in RGB color space)
lower = np.array([100, 100, 100])
upper = np.array([255, 255, 255])

# Create a VideoCapture object to read from the webcam
cap = cv2.VideoCapture(0)

while True:
    # Capture a frame from the webcam
    ret, frame = cap.read()
    
    # Convert the frame from BGR to RGB color space
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    
    # Apply the color threshold to extract the plastic bottle regions
    mask = cv2.inRange(frame_rgb, lower, upper)
    contours, hierarchy = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    # Make predictions for each detected region
    for contour in contours:
        (x, y, w, h) = cv2.boundingRect(contour)
        roi = frame[y:y+h, x:x+w]
        roi_resized = cv2.resize(roi, (100, 100))
        roi_flattened = roi_resized.reshape(1, -1).astype('float32') / 255
        prediction = model.predict(roi_flattened)
        
        # Draw a rectangle around the detected plastic bottle and label it as "Plastic" or "Non-Plastic"
        if prediction == 'Plastic':
            cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)
            cv2.putText(frame, 'Plastic', (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)
        
    
    # Display the frame with the detected regions and predictions
    cv2.imshow('Plastic Detection', frame)
    
    # Exit the program if the user presses the 'q' key
    if cv2.waitKey(1) == ord('q'):
        break

# Release the VideoCapture object and close all windows
cap.release()
cv2.destroyAllWindows()


Single Snap

In [11]:
import cv2
import joblib

# Load the saved SVM model
model = joblib.load('PlasticDetection.pkl')

# Set the webcam source
cap = cv2.VideoCapture(0)

# Capture an image from the webcam
ret, frame = cap.read()

# Resize the image to 100x100
resized_frame = cv2.resize(frame, (100, 100))

# Flatten the image and normalize the pixel values
flattened_frame = resized_frame.reshape(1, -1).astype('float32') / 255

# Pass the image through the model for prediction
prediction = model.predict(flattened_frame)

# Draw a bounding box around the detected plastic bottle
if prediction == 'plastic':
    cv2.rectangle(frame, (0, 0), (frame.shape[1], frame.shape[0]), (0, 255, 0), thickness=2)

# Display the image and the predicted label
cv2.imwrite('test.jpg',frame)
image = cv2.imread("test.jpg")
cv2.imshow(str(prediction), image)
print('Prediction:', prediction)
cv2.waitKey(0)

# Release the webcam source and close the window
cap.release()
cv2.destroyAllWindows()


Prediction: ['Non Plastic']


In [12]:
import cv2

# Set the webcam source
cap = cv2.VideoCapture(1)
model = joblib.load('PlasticDetection.pkl')

# Capture an image from the webcam
ret, frame = cap.read()

# Release the webcam source
cap.release()

# Save the captured image to a file
cv2.imwrite('captured_image.jpg', frame)

# Display the captured image
cv2.imshow('image', frame)
cv2.waitKey(0)
cv2.destroyAllWindows()


QR code Creation

In [3]:
import qrcode

reward_amount = 3000

qr = qrcode.QRCode(version=1, box_size=10, border=5)
data = {'Amount':reward_amount,'Id':1}
qr.add_data(data)
qr.make(fit=True)
img = qr.make_image(fill_color='black', back_color='white')
img.show()

Scan QR code

In [6]:
import cv2

cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    d = cv2.QRCodeDetector()
    val,c,f=d.detectAndDecode(frame)

    cv2.imshow('QR Code Scanner', frame)

    if val:
        print(val)
        break

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


{'Amount': 3000, 'Id': 1}
